In [207]:
!uv pip install navec pymorphy2 pandas

Audited 3 packages in 5ms


In [208]:
from navec import Navec
import pandas as pd
import pymorphy2

# существительные, нам понадобиться только колонка с начальной формой – bare
# https://github.com/Badestrand/russian-dictionary
nouns = pd.read_csv('nouns.csv', sep='\t')[['bare']]
# переименуем колонку для наглядности
nouns = nouns.rename(columns={"bare": "word"})

# эмбеддинги
# https://github.com/natasha/navec
navec = Navec.load('navec_hudlit_v1_12B_500K_300d_100q.tar')

# морфологический анализ
morph = pymorphy2.MorphAnalyzer()

In [209]:
def is_proper_noun(word):
    parsed = morph.parse(word)[0]
    # есть в датасете эмбеддингов
    if (word not in navec):
        return False
    # фильтруем географические названия
    if ('Geox' in parsed.tag):
        return False
    # фильтруем имена
    if ('Name' in parsed.tag):
        return False
    # фильтруем фамилии
    if ('Surn' in parsed.tag):
        return False
    # убеждаемся, что точно существительное
    return 'NOUN' == parsed.tag.POS

nouns = nouns[nouns['word'].apply(is_proper_noun)]
nouns = nouns.sort_values('word').drop_duplicates('word', keep='last')
nouns

,word
4362,абажур
15031,аббат
23975,аббатиса
15032,аббатство
11668,аббревиатура
...,...
4963,ёжик
2045,ёлка
6075,ёлочка
3660,ёмкость


In [210]:
def to_vec(word):
    return navec[word]

nouns['embeddings'] = nouns['word'].apply(to_vec)
nouns

,word,embeddings
4362,абажур,"[0.49163, 0.003914773, -0.034092356, -0.338077..."
15031,аббат,"[-0.4445479, 0.24872686, -0.16423002, -0.00452..."
23975,аббатиса,"[-0.17017908, 0.67665476, 0.21191145, -0.25886..."
15032,аббатство,"[-0.089431904, 0.39096224, -0.19686256, 0.3650..."
11668,аббревиатура,"[0.036720417, 0.92295927, 0.37911993, -0.89247..."
...,...,...
4963,ёжик,"[0.5135839, 0.48293516, 0.51696765, 0.1938662,..."
2045,ёлка,"[0.57082283, -0.02571172, 0.20820206, -0.17096..."
6075,ёлочка,"[0.7598189, 0.5453649, -0.112766445, -0.277389..."
3660,ёмкость,"[0.22543699, -0.39721358, 0.6805563, 0.3375504..."


In [211]:
# если хочется выбрать конкретное
# secret_word = nouns[nouns['word'] == 'деньги']
secret_word = nouns.sample(n=1)
secret_word

,word,embeddings
4506,искренность,"[-0.1538672, 0.18462132, 0.00536857, 0.1487121..."


In [212]:
import math

def cosine_similarity(vec1, vec2):
    dot_product = sum(v1 * v2 for v1, v2 in zip(vec1, vec2))
    magnitude_vec1 = math.sqrt(sum(v1**2 for v1 in vec1))
    magnitude_vec2 = math.sqrt(sum(v2**2 for v2 in vec2))

    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0

    return dot_product / (magnitude_vec1 * magnitude_vec2)

secret_word_vec = secret_word['embeddings'].values[0]
nouns['cosine_similarity'] = nouns['embeddings'].apply(lambda emb: cosine_similarity(secret_word_vec, emb))
nouns = nouns.sort_values(by='cosine_similarity', ascending=False).reset_index(drop=True)
nouns

,word,embeddings,cosine_similarity
0,искренность,"[-0.1538672, 0.18462132, 0.00536857, 0.1487121...",1.000000
1,правдивость,"[-0.4352695, 0.5312125, 0.2214203, 0.01515189,...",0.696473
2,честность,"[-0.4445479, 0.24872686, -0.16423002, 0.149342...",0.696473
3,откровенность,"[0.11729202, -0.050626393, 0.29470918, 0.11379...",0.671834
4,сердечность,"[0.28366756, 0.008455998, 0.36166316, -0.05277...",0.659590
...,...,...,...
19869,танцкласс,"[0.52534044, 0.19559065, 0.09048509, -0.145310...",-0.221405
19870,сонник,"[-0.097243235, -0.15211679, 0.44589067, 0.1137...",-0.222282
19871,кобчик,"[0.1168642, 0.4645877, -0.17707707, 0.21666782...",-0.222360
19872,паёк,"[0.057401087, -0.69955, 0.550758, -0.08534745,...",-0.223937


In [214]:
from datetime import datetime

nouns['rank'] = nouns.index + 1
nouns['date'] = datetime.today().strftime('%Y-%m-%d')
nouns

,word,embeddings,cosine_similarity,rank,date
0,искренность,"[-0.1538672, 0.18462132, 0.00536857, 0.1487121...",1.000000,1,2025-08-04
1,правдивость,"[-0.4352695, 0.5312125, 0.2214203, 0.01515189,...",0.696473,2,2025-08-04
2,честность,"[-0.4445479, 0.24872686, -0.16423002, 0.149342...",0.696473,3,2025-08-04
3,откровенность,"[0.11729202, -0.050626393, 0.29470918, 0.11379...",0.671834,4,2025-08-04
4,сердечность,"[0.28366756, 0.008455998, 0.36166316, -0.05277...",0.659590,5,2025-08-04
...,...,...,...,...,...
19869,танцкласс,"[0.52534044, 0.19559065, 0.09048509, -0.145310...",-0.221405,19870,2025-08-04
19870,сонник,"[-0.097243235, -0.15211679, 0.44589067, 0.1137...",-0.222282,19871,2025-08-04
19871,кобчик,"[0.1168642, 0.4645877, -0.17707707, 0.21666782...",-0.222360,19872,2025-08-04
19872,паёк,"[0.057401087, -0.69955, 0.550758, -0.08534745,...",-0.223937,19873,2025-08-04


In [215]:
result = nouns[['word', 'rank', 'date']]
result.to_csv('words.csv', index=False)